In [ ]:
from pyspark.sql.types import *

In [ ]:
import pyspark.sql.functions as f

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql.types import StructField,IntegerType, FloatType, BooleanType ,StructType,StringType

In [ ]:
# definição do esquema da tabela temporária de contratos
CamposContr=[StructField('contract_id',IntegerType(),True),
             StructField('client_id',IntegerType(),True),
             StructField('client_name',StringType(),True),
             StructField('percentage',FloatType(),True),
             StructField('is_active',BooleanType(),True)
       ]
ContratosStruct=StructType(fields=CamposContr)

In [ ]:
spark = (
    SparkSession
    .builder
    .master("local")
    .appName("test-GetNet")
    .config("spark.sql.shuffle.partitions", 13)
    .config("spark.driver.memory", "10g")
    .config("setLogLevel", "INFO")
    .config("spark.logConf", "true")
    .getOrCreate()
)
#spark.sparkContext.setLogLevel("WARN")
  
    


In [ ]:
# leitura do csv de contratos
df_contratos = spark.read.csv('contratos.csv', header=True, sep=';',schema=ContratosStruct)

In [ ]:
# transformação para tabela temporária de contratos
df_contratos.createOrReplaceTempView("Contratos")

In [ ]:
# definição do esquema da tabela temporária de transações
spark.sql("SELECT * FROM Contratos where is_active is true limit 3").show()

In [ ]:
# definição do esquema da tabela temporária de transações
CamposTrans=[StructField('transaction_id',IntegerType(),True),
             StructField('client_id',IntegerType(),True),
              StructField('total_amount',FloatType(),True),
             StructField('discount_percentage',FloatType(),True)
       ]
TransacoesStruct=StructType(fields=CamposTrans)

In [ ]:
# leitura do csv de transações
df_transacoes = spark.read.csv('transacoes*.csv', header=True, sep=';', schema=TransacoesStruct)

In [ ]:
# transformação para tabela temporária de transações
df_transacoes.createOrReplaceTempView("Transacoes")

In [ ]:
# leitura da base de transações
spark.sql("SELECT * FROM Transacoes limit 3").show()

In [ ]:
# valor de cada transação
spark.sql("SELECT transaction_id, c.client_id, client_name, contract_id, percentage, total_amount, (total_amount-(total_amount*coalesce(discount_percentage,0)/100.00))*percentage/100.00 val_trans, (total_amount-(total_amount*coalesce(discount_percentage,0)/100.00))*percentage/100.00 val_trans_serv_loc_cart_cred FROM Transacoes t left join Contratos c on (c.client_id=t.client_id) where is_active is true limit 3").show()

In [ ]:
# valor total de todas as transações
spark.sql("SELECT sum((total_amount-(total_amount*coalesce(discount_percentage,0)/100.00))*percentage/100.00) val_tot_serv_loc_cart_cred FROM Transacoes t left join Contratos c on (c.client_id=t.client_id) where is_active is true").show()